In [24]:
import pandas as pd
from  sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
smile_data=pd.read_csv('smile_data.csv')

In [27]:
smile_data.head()

,smiles,target1,target2,target3,target4,target5,target6,target7,target8,target9,target10,target11,target12
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,CCN1C(=O)NC(c2ccccc2)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [5]:
smile_data_copy=smile_data[["smiles","target1"]]

In [6]:
smile_data_copy.shape

(8014, 2)

In [7]:
smile_data_copy_no_miss=smile_data_copy.dropna(how='any',axis=0)

In [8]:
smile_data_copy_no_miss.target1.value_counts()

0.0    7129
1.0     310
Name: target1, dtype: int64

In [11]:
mol=smile_data['smiles']

In [12]:
countVectorizer = CountVectorizer()

In [13]:
textVector = countVectorizer.fit_transform(mol)

In [14]:
textVector

<8014x3168 sparse matrix of type '<class 'numpy.int64'>'
	with 28451 stored elements in Compressed Sparse Row format>

In [15]:
textVector.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
text_vec_mat=textVector.todense()

In [17]:
text_vec_df=pd.DataFrame(text_vec_mat,index=None)

In [18]:
text_vec_df_combine_org=pd.concat([smile_data_copy,text_vec_df],axis=1)

In [19]:
text_vec_df_combine_org.shape

(8014, 3170)

In [20]:
text_vec_df_combine_org_nomiss=text_vec_df_combine_org.dropna(how='any')

In [21]:
text_vec_df_combine_org_nomiss.head()

,smiles,target1,0,1,2,3,4,5,6,7,...,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CCN1C(=O)NC(c2ccccc2)C1=O,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df=text_vec_df_combine_org_nomiss

In [23]:
df.head()

,smiles,target1,0,1,2,3,4,5,6,7,...,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CCN1C(=O)NC(c2ccccc2)C1=O,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:

def build_dnn(layers_num):#特征个数，128*128
    model_bp_ks = keras.Sequential()
    model_bp_ks.add(Dense(layers_num[1], input_shape=(layers_num[0],)))
    # model_bp_ks.add(regularizers.l1(0.0001))
    model_bp_ks.add(Activation('relu'))
    model_bp_ks.add(Dropout(0.5))

    model_bp_ks.add(Dense(layers_num[2]))
    # model_bp_ks.add(regularizers.l1(0.001))
    model_bp_ks.add(Activation('relu'))
    model_bp_ks.add(Dropout(0.5))

    # model_bp_ks.add(Dense(1))
    # model_bp_ks.add(Activation('linear'))

    model_bp_ks.add(Dense(1))
    model_bp_ks.add(Activation('sigmoid'))
    model_bp_ks.summary()  # print每层的参数
    # model_bp_ks.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    # model_bp_ks.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    # model_bp_ks.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model_bp_ks.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model_bp_ks
df = df.dropna(how = "any",axis = 0)
feat = [x for x in df.columns.tolist() if x not in ("smiles","target1")]
x_train = df[feat]
y_train = df["target1"]
x_train_used, x_validation, y_train_used, y_validation = train_test_split(x_train,
                                                                                  y_train,
                                                                                  test_size=0.4,
                                                                                  random_state=1)

estimators = build_dnn([x_train_used.shape[1], 128, 128])
# estimators = build_dnn([x_train_used.shape[1], 128, 128])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

estimators.fit(x_train_used,
               y_train_used,
               batch_size=32,
               epochs=100,
               verbose=2,
               # shuffle=False,
               validation_data=(x_validation, y_validation),
               callbacks=[early_stopping]
               # validation_split=0.05
               )
               
y_pred_train_all_prob = estimators.predict_proba(x_train_used)
best_threshold = 0.3
y_pred_label_train = np.where(y_pred_train_all_prob > best_threshold, 1, 0)

def evaluation(y_real, y_pred):
    accuracy_ = accuracy_score(y_real, y_pred)
    precision_ = precision_score(y_real, y_pred)
    recall_score_ = recall_score(y_real, y_pred)
    f1_score_ = f1_score(y_real, y_pred)
    # auc_ = metrics.roc_curve(y_test.values, ypred_label, pos_label=1)
    fpr, tpr, thresholds = metrics.roc_curve(y_real, y_pred, pos_label=1)
    auc_ = metrics.auc(fpr, tpr)
    return accuracy_, precision_, recall_score_, auc_, f1_score_

accuracy_t, precision_t, recall_score_t, auc_, f1_score_t = evaluation(y_train_used, y_pred_label_train)
performance_train=pd.DataFrame()
performance_train.loc[0, "accuracy"] = accuracy_t
performance_train.loc[0, "precision"] = precision_t
performance_train.loc[0, "recall_score"] = recall_score_t
# performance_train_df.loc[i, "auc"] = auc_t
performance_train.loc[0, "f1_score"] = f1_score_t

print("accuracy:{} precision:{} recall_score:{} f1_score:{} "
      .format(accuracy_t, precision_t, recall_score_t, f1_score_t))

y_pred_train_all_prob = estimators.predict_proba(x_validation)
best_threshold = 0.5
y_pred_label_train = np.where(y_pred_train_all_prob > best_threshold, 1, 0)
      
accuracy_t, precision_t, recall_score_t, auc_, f1_score_t = evaluation(y_validation, y_pred_label_train)
performance_train.loc[1, "accuracy"] = accuracy_t
performance_train.loc[1, "precision"] = precision_t
performance_train.loc[1, "recall_score"] = recall_score_t
# performance_train_df.loc[i, "auc"] = auc_t
performance_train.loc[1, "f1_score"] = f1_score_t

print("accuracy:{} precision:{} recall_score:{} f1_score:{} "
      .format(accuracy_t, precision_t, recall_score_t, f1_score_t))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               405632    
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
__________